In [1]:
from cosecha_colectiva import miscelaneous as m
from cosecha_colectiva import config as config
m.try_find_conf_file()


from cosecha_colectiva import miscelaneous as m

import logging
import pathlib
from datetime import datetime
import datajoint as dj
import time
import numpy as np

import pandas as pd


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
grupo_inicial= 0

In [3]:
def number_to_excel_column(n):
    result = ''
    while n > 0:
        remainder = (n - 1) % 26
        result = chr(ord('A') + remainder) + result
        n = (n - 1) // 26
    return result

In [4]:
cosecha_db = dj.create_virtual_module(config.db_name, config.db_name)

[2025-05-17 12:50:28,980][INFO]: Connecting root@junction.proxy.rlwy.net:19994
[2025-05-17 12:50:30,076][INFO]: Connected root@junction.proxy.rlwy.net:19994


## Ingresa datos de Grupos

In [5]:
ogsd = cosecha_db.OldGruposStatsDashboard()
table_attributes = pd.DataFrame.from_dict(ogsd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'A:'+number_to_excel_column(len(this_cols))
columns_read

'A:BN'

In [6]:
for i in range(grupo_inicial, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStats',
                          skiprows=30, nrows=1, names=this_cols, header=None, dtype=this_dtypes
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    ogsd.insert(dict_data, skip_duplicates=True)
    time.sleep(0.1)


0
1
2


## Lee datos para las otras tablas (num_socios, num_sesiones, nombre_grupo)

In [6]:
num_socios_grupo = [] 
Grupos_Names = []
num_sesiones_grupo = []
num_sesiones_socios_grupo = []
for i in range(grupo_inicial, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    data_num_socios = pd.read_excel(url,
                          sheet_name='DatosParaStats',
                        nrows=3, usecols='A:B', header=None
                  )
    num_socios_grupo.append(int(data_num_socios.loc[0,1]))
    Grupos_Names.append(data_num_socios.loc[1,1])
    data_num_sesiones = pd.read_excel(url,
                          sheet_name='DatosParaStats',
                          skiprows=59, nrows=4, usecols='A:C', header=None
                  )
    num_sesiones_grupo.append(int(data_num_sesiones.loc[0,2]))

    num_sesiones_socios_grupo.append(int(data_num_sesiones.loc[3,2]))



0
1
2


## Ingresa datos Socios

In [6]:
ossd = cosecha_db.OldSociosStatsDashboard()
table_attributes = pd.DataFrame.from_dict(ossd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'A:'+number_to_excel_column(len(this_cols))
columns_read

'A:BI'

In [7]:
idx_grupo = 0
for i in range(grupo_inicial, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    print(Grupos_Names[idx_grupo])
    print('num_socios_grupo[idx_grupo]', num_socios_grupo[idx_grupo])
    print('num_sesiones_grupo[idx_grupo]', num_sesiones_grupo[idx_grupo])
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStats',
                          skiprows=1, nrows=num_socios_grupo[idx_grupo], names=this_cols, usecols=columns_read, dtype=this_dtypes, header=None
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    ossd.insert(dict_data, skip_duplicates=True)
    idx_grupo += 1
    time.sleep(0.1)


0
La Luz
num_socios_grupo[idx_grupo] 12
num_sesiones_grupo[idx_grupo] 92
1
Abejas
num_socios_grupo[idx_grupo] 11
num_sesiones_grupo[idx_grupo] 78
2
Hidalgo
num_socios_grupo[idx_grupo] 12
num_sesiones_grupo[idx_grupo] 68


## Ingresa datos Sesiones

In [10]:
oSeSd = cosecha_db.OldSesionesStatsDashboard()
table_attributes = pd.DataFrame.from_dict(oSeSd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'D:'+number_to_excel_column(len(this_cols)+3)
columns_read

'D:BM'

In [11]:
idx_grupo = 0
for i in range(grupo_inicial, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    print(Grupos_Names[idx_grupo])
    print('num_socios_grupo[i]', num_socios_grupo[idx_grupo])
    print('num_sesiones_grupo[i]', num_sesiones_grupo[idx_grupo])
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStatsSesiones',
                          skiprows=200, nrows=num_sesiones_grupo[idx_grupo], names=this_cols, usecols=columns_read, header=None, dtype=this_dtypes
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    oSeSd.insert(dict_data, skip_duplicates=True)
    idx_grupo +=1
    time.sleep(0.1)


0
La Luz
num_socios_grupo[i] 12
num_sesiones_grupo[i] 92
1
Abejas
num_socios_grupo[i] 11
num_sesiones_grupo[i] 78
2
Hidalgo
num_socios_grupo[i] 12
num_sesiones_grupo[i] 68


## Ingresa datos Sesiones-Socio

In [7]:
osssd = cosecha_db.OldSesionesSociosStatsDashboard()
table_attributes = pd.DataFrame.from_dict(osssd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'D:'+number_to_excel_column(len(this_cols)+3)
columns_read

'D:BS'

In [8]:
idx_grupo = 0
for i in range(grupo_inicial, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    print(Grupos_Names[idx_grupo])
    print('num_socios_grupo[i]', num_socios_grupo[idx_grupo])
    print('num_sesiones_grupo[i]', num_sesiones_grupo[idx_grupo])
    print('num_sesiones_socios_grupo[idx_grupo]', num_sesiones_socios_grupo[idx_grupo])
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStatsSociosSesiones',
                          skiprows=1500, nrows=num_sesiones_socios_grupo[idx_grupo], names=this_cols, usecols=columns_read, header=None, dtype=this_dtypes
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    osssd.insert(dict_data, skip_duplicates=True)
    idx_grupo +=1
    time.sleep(0.1)


0
La Luz
num_socios_grupo[i] 12
num_sesiones_grupo[i] 92
num_sesiones_socios_grupo[idx_grupo] 1104
1
Abejas
num_socios_grupo[i] 11
num_sesiones_grupo[i] 78
num_sesiones_socios_grupo[idx_grupo] 858
2
Hidalgo
num_socios_grupo[i] 12
num_sesiones_grupo[i] 68
num_sesiones_socios_grupo[idx_grupo] 816


In [9]:
df_data

,nombre_grupo,num_sesion_socio,num_sesion,num_socio,Sesion_Socio_id,Sesion_id,Socio_id,Fecha,Activa,Caja,...,sum_ganancias_entregadas,num_ganancias,sesion_id_interes_prestamo,socio_id_interes_prestamo,sum_interes,num_prestamos_activos,num_prestamos_vigentes,num_prestamos_morosos,num_prestamos_ampliados,num_prestamos_ampliados_morosos
0,Hidalgo,1,1,1,4001,200,40,2019-03-01,0,100.0,...,0.000000,0,200,40,0.0,0,0,0,0,0
1,Hidalgo,2,1,2,4002,200,41,2019-03-01,0,100.0,...,0.000000,0,200,41,0.0,0,0,0,0,0
2,Hidalgo,3,1,3,4003,200,42,2019-03-01,0,100.0,...,0.000000,0,200,42,0.0,0,0,0,0,0
3,Hidalgo,4,1,4,4004,200,43,2019-03-01,0,100.0,...,0.000000,0,200,43,0.0,0,0,0,0,0
4,Hidalgo,5,1,5,4005,200,44,2019-03-01,0,100.0,...,0.000000,0,200,44,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,Hidalgo,812,68,8,4812,267,47,2024-12-01,0,21563.0,...,89.034596,0,267,47,366.0,1,1,0,0,0
812,Hidalgo,813,68,9,4813,267,48,2024-12-01,0,-27465.0,...,106.206263,0,267,48,114.0,1,1,0,0,0
813,Hidalgo,814,68,10,4814,267,49,2024-12-01,0,0.0,...,0.000000,0,267,49,0.0,0,0,0,0,0
814,Hidalgo,815,68,11,4815,267,50,2024-12-01,0,1800.0,...,104.518873,0,267,50,350.0,1,1,0,0,0
